In [3]:
import pandas as pd
import numpy as np 


In [11]:
data_puestos = pd.read_pickle("matcher/static/data/puestos.pickle")
data_puestos

,PUESTO
0,Ejecutivo Ventas CALL CENTER C/EXP PRESENCIAL ...
1,Jefe de Selección del Talento
2,Administrador de Cuenta Comercial - Villa El S...
3,Analista Sr. Comunicación Organizacional
4,Mecanico Aeronautico - Latam Airlines Chile
...,...
12654,Asesor comercial - Campo
12655,Atención al Cliente Sin Experiencia Turno Tard...
12656,ANALISTA DE COSTOS
12657,Asistente Contable - Tributaciones


In [ ]:
# Limpieza de datos
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')


def limpiar_texto(texto):
    """
    Elimina todos los caracteres que no sean alfanuméricos (letras y números)
    """
    texto = texto.lower()
    return re.sub(r'[^a-zA-Z0-9] ', ' ', texto)

def tokenizar_espanol(texto):
    """
    Tokeniza texto en español usando NLTK
    """
    # Tokenizar en palabras
    tokens = word_tokenize(texto, language='spanish')
    
    return tokens

def quitar_stopwords(tokens):
    """
    Elimina las stop words en español de una lista de tokens
    """
    stop_words = set(stopwords.words('spanish'))
    tokens_filtrados = [token for token in tokens if token.lower() not in stop_words]
    return tokens_filtrados




nlp = spacy.load('es_core_news_sm')

def lematizar_spacy(texto):
    """
    Lematiza texto usando spaCy (más preciso que stemming)
    """
    texto = ' '.join(texto)
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    texto_final = ' '.join(lemas)
    return texto_final

def tratamiento_texto(texto):
    texto = limpiar_texto(texto)
    texto = tokenizar_espanol(texto)
    texto = quitar_stopwords(texto)
    texto = lematizar_spacy(texto)
    return texto

vect = CountVectorizer()
matriz_puestos = vect.fit_transform(data_puestos["PUESTO"].apply(tratamiento_texto))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adrianalarcon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adrianalarcon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
## Hacer el tratamiento para el cv
import PyPDF2
def leer_pdf_pypdf2(ruta_archivo):
    """
    Lee el texto de un archivo PDF usando PyPDF2
    """
    texto_completo = ""
    
    with open(ruta_archivo, 'rb') as archivo:
        lector = PyPDF2.PdfReader(archivo)
        
        # Iterar por todas las páginas
        for pagina in lector.pages:
            texto_completo += pagina.extract_text() + "\n"
    
    return texto_completo

# Uso:
# pip install PyPDF2
texto = leer_pdf_pypdf2("documents/cv_desarrollador_software.pdf_2025_8_26.pdf")
df = pd.DataFrame()
df["texto"] = [texto]
matriz_cv = vect.transform(df["texto"].apply(tratamiento_texto))

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

indices = pd.Series(cosine_similarity(matriz_puestos,matriz_cv)[:,0]).sort_values(ascending=False).head(10).index

data_puestos.iloc[indices, :]

,PUESTO
10269,Practicante Profesional de Desarrollo de Software
11987,ASISTENTE DE DESARROLLO TI – RIESGOS Y GESTIÓN
11990,ASISTENTE DE DESARROLLO TI – RIESGOS Y GESTIÓN
355,PRACTICANTE EN DESARROLLO WEB
3839,Practicante Profesional: Sistema de Gestión
775,PRACTICANTE EN DESARROLLO WEB - CAJAMARCA
5049,Ingeniero de Desarrollo de Sistemas Contra Inc...
11174,Practicante de Sistema Integrado de Gestión
8641,Asistente de Gestión y Desarrollo Humano - Pro...
6076,Practicante Profesional de Sistemas de Gestió...


In [26]:
import pickle


with open('vectorizer.pickle', 'wb') as f:
    pickle.dump(vect, f)